In [2]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
import statsmodels.api as sm
from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier
from statsmodels.iolib.summary2 import summary_col
import warnings
warnings.filterwarnings("ignore")

In [3]:
df = pd.read_csv("/Users/Giulio/Library/Mobile Documents/com~apple~CloudDocs/Bocconi/Third Year/Exchange/Courses/Labor Economics/Extra credit paper/Datasets/train.csv")

In [4]:
df.rename(columns={"educational-num":"schooling", "race":"ethnicity", "hours-per-week":"hours", "native-country":"immigrant", "income_>50K":"target"}, inplace=True) 

In [5]:
df.drop(["education", "relationship", "capital-gain", "capital-loss"], axis=1, inplace=True)

In [6]:
print(len(df[df.isnull().any(axis=1)]) / len(df))
df.dropna(inplace=True)

0.07348090179038606


In [7]:
df["workclass"] = df["workclass"].replace({"State-gov": "Gov", "Federal-gov": "Gov", "Local-gov": "Gov", 
                                           "Self-emp-not-inc": "Self-emp", "Self-emp-inc": "Self-emp"})

df["marital-status"] = df["marital-status"].replace({"Divorced": "Separated", "Married-spouse-absent": "Separated",
                                                    "Married-civ-spouse": "Married", "Married-AF-spouse": "Married"})

df["occupation"] = df["occupation"].replace({"Exec-managerial": "White Collar", 'Adm-clerical': 'White Collar', 'Prof-specialty': 'White Collar', 'Tech-support': 'White Collar', 
                                             'Craft-repair': 'Blue Collar', 'Machine-op-inspct': 'Blue Collar', 'Handlers-cleaners': 'Blue Collar', 'Transport-moving': 'Blue Collar','Farming-fishing': 'Blue Collar',
                                             'Sales': 'Service', 'Other-service': 'Service', 'Protective-serv': 'Service', 'Priv-house-serv': 'Service',
                                             'Armed-Forces': 'Military'})

df["ethnicity"] = df["ethnicity"].replace({"Asian-Pac-Islander": "Asian", 
                                           "Amer-Indian-Eskimo": "Eskimo"})

df["immigrant"] = np.where(df["immigrant"] == "United-States", 0, 1)


In [8]:
print(len(df[df["occupation"]=="Military"]) / len(df))
df = df[df["occupation"] != "Military"]

0.0002946448302109166


In [9]:
# Standardization
features_to_standardize = ["age", "fnlwgt", "schooling", "hours"]
for i in range(len(features_to_standardize)):
    x = np.array(df[features_to_standardize[i]]).reshape(-1,1)
    scaler = StandardScaler()
    scaler.fit(x)
    df[features_to_standardize[i]] = scaler.transform(x)

In [10]:
# One-Hot encoding
features_to_encode = ["workclass", "marital-status", "occupation", "ethnicity"]
other_features = ["age", "fnlwgt", "schooling", "gender", "hours", "immigrant", "target"]

categorical_data = df[features_to_encode]

encoder = OneHotEncoder(sparse=False, drop="first")

encoded_data = encoder.fit_transform(categorical_data)
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names(features_to_encode), index=df.index)
df_ohe = pd.concat([encoded_df, df[other_features]], axis=1)

In [11]:
# Simple encoding
for i in range(len(features_to_encode)):
    x = df[features_to_encode[i]]
    encoded_data, unique = pd.factorize(x)
    df[features_to_encode[i]] = encoded_data

In [12]:
df_men = df[df["gender"]=="Male"]
df_women = df[df["gender"]=="Female"]
df_men.drop(["gender"], axis=1, inplace=True)
df_women.drop(["gender"], axis=1, inplace=True)

df_ohe_men = df_ohe[df_ohe["gender"]=="Male"]
df_ohe_women = df_ohe[df_ohe["gender"]=="Female"]
df_ohe_men.drop(["gender"], axis=1, inplace=True)
df_ohe_women.drop(["gender"], axis=1, inplace=True)

## Logistic Regression

In [13]:
def analyze_lg(df):
    X = df.drop("target", axis=1)
    y = df["target"]
    X = sm.add_constant(X) # here I just add a constant for statsmodels
    model = sm.Logit(y, X).fit()
    print(model.summary())

analyze_lg(df_ohe_men)
analyze_lg(df_ohe_women)

Optimization terminated successfully.
         Current function value: 0.438515
         Iterations 7
                           Logit Regression Results                           
Dep. Variable:                 target   No. Observations:                27478
Model:                          Logit   Df Residuals:                    27460
Method:                           MLE   Df Model:                           17
Date:                Sat, 27 Apr 2024   Pseudo R-squ.:                  0.2939
Time:                        14:20:22   Log-Likelihood:                -12050.
converged:                       True   LL-Null:                       -17065.
Covariance Type:            nonrobust   LLR p-value:                     0.000
                                   coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------------
const                           -0.9162      0.117     -7.806      0.000

## Random Forest

In [14]:
X_men_rf = df_men.drop('target', axis=1)
y_men_rf = df_men['target']
X_women_rf = df_women.drop('target', axis=1)
y_women_rf = df_women['target']

rf_men = RandomForestClassifier(n_estimators=100)
rf_men.fit(X_men_rf, y_men_rf)

rf_women = RandomForestClassifier(n_estimators=100)
rf_women.fit(X_women_rf, y_women_rf)

importances_men = rf_men.feature_importances_
importances_women = rf_women.feature_importances_
importances_difference = rf_men.feature_importances_ - rf_women.feature_importances_
importances_average = (rf_men.feature_importances_ + rf_women.feature_importances_) / 2

feature_comparison = pd.DataFrame({
    "Feature": X_men_rf.columns,
    "Importance_Men": importances_men * 100,
    "Importance_Women": importances_women * 100,
    "Difference": importances_difference * 100,
    "Average": importances_average * 100
})

feature_comparison.sort_values(by="Average", ascending=False)

,Feature,Importance_Men,Importance_Women,Difference,Average
2,fnlwgt,30.259871,24.680234,5.579637,27.470053
0,age,21.641110,19.845440,1.795670,20.743275
4,marital-status,11.757529,21.264358,-9.506829,16.510944
3,schooling,14.110531,12.741629,1.368901,13.426080
7,hours,10.680897,11.250375,-0.569477,10.965636
5,occupation,5.836825,3.538922,2.297903,4.687874
1,workclass,2.959744,3.133205,-0.173461,3.046474
6,ethnicity,1.750908,2.335046,-0.584137,2.042977
8,immigrant,1.002584,1.210791,-0.208207,1.106687


## Fairlie Decomposition Analysis

In [15]:
X_men = df_ohe_men.drop(["target"], axis=1)
X_women = df_ohe_women.drop(["target"], axis=1)
y_men = np.array(df_ohe_men["target"])
y_women = np.array(df_ohe_women["target"])

cols = list(X_men.columns)

model_men = LogisticRegression().fit(X_men, y_men)

effects = []
for i in range(1000):
    X_women_shuffled = shuffle(X_women)
    X_men_shorter = X_men.sample(len(X_women), replace=False)
    delta_p = 0
    for j in range(len(cols)):
        X_temp = X_men_shorter.copy()
        X_temp[cols[j]] = np.array(X_women_shuffled[cols[j]])
        p_new = model_men.predict_proba(X_temp)[:, 1]
        delta_p += np.mean(model_men.predict_proba(X_men_shorter)[:, 1] - p_new)
    effects.append(delta_p / X_men.shape[1])

average_explainable_effect = np.mean(effects)
average_explainable_effect

0.008982802157304935

In [16]:
model_women = LogisticRegression().fit(X_women, y_women)
average_diff = np.mean(model_men.predict_proba(X_men)[:,1])  - np.mean(model_women.predict_proba(X_women)[:,1])    
average_diff

0.19881043464790613

In [17]:
print(f"""
The average predicted probability of men earning more than $50,000 a year is {round(average_diff*100, 2)}% higher than for women          
According to the data and models employed here, observed characteristics only account for {round(average_explainable_effect*100, 2)}% of this difference        
This means that the rest, equal to {round((average_diff-average_explainable_effect)*100, 2)}% is left unexplained, possibly due to discrimination
""")


The average predicted probability of men earning more than $50,000 a year is 19.88% higher than for women          
According to the data and models employed here, observed characteristics only account for 0.9% of this difference        
This means that the rest, equal to 18.98% is left unexplained, possibly due to discrimination

